## **remap**

* `remap` implements a **general pixel‐based warping**: For every pixel in the **destination (output)** image located at `(u, v)`,
OpenCV looks up the corresponding pixel in the **source** image:

$$
dst(v, u) = src(map_y(v, u),\ map_x(v, u))
$$

That means:

* `map_x(v,u)` → the **x-coordinate (column)** in the **source**
* `map_y(v,u)` → the **y-coordinate (row)** in the **source**

So, you must think **inverse mapping**:

> For each output pixel `(u,v)`, tell me **where in the source** it comes from.


##  **OpenCV Two Coordinate Conventions**

OpenCV uses **two coordinate conventions** that look similar but differ in **semantic meaning**:

| Concept                              | Symbol                      | Meaning                                        |
| ------------------------------------ | --------------------------- | ---------------------------------------------- |
| **Matrix access**                    | `mat.at<type>(row, column)` | Access by **(y, x)** = **(row, column)** order |
| **Geometric coordinate (cv::Point)** | `Point(x, y)`               | Access by **(u, v)** = **(column, row)** order |

These two map as:

$$
u = \text{column} = x, \quad v = \text{row} = y
$$

and both access the **same pixel**, because OpenCV defines:

```cpp
mat.at<type>(Point(x, y)) == mat.at<type>(y, x)
```

---

### Visual Layout

Your ASCII diagrams are absolutely correct — they reflect how images are stored and addressed in OpenCV memory:

#### **Geometric (u,v) coordinate space**

```cpp
              u (x = column)
    ------------------------------------------►
    | (0,0) (1,0) (2,0) (3,0)
    | (0,1) (1,1) (2,1) (3,1)
    | (0,2) (1,2) (2,2) (3,2)
  v | 
    |
    ▼
      (y = row)
```

This is the **image-space coordinate system**, consistent with:

* `Point(u, v)`
* `cv::circle(image, Point(u, v), ...)`
* `remap`, `warpAffine`, `warpPerspective`, etc.

---

### **Matrix (row, column) indexing**

```cpp
    column (x)
    -----------►
    | mat.at<type>(0,0) mat.at<type>(0,1) mat.at<type>(0,2) mat.at<type>(0,3)
row | mat.at<type>(1,0) mat.at<type>(1,1) mat.at<type>(1,2) mat.at<type>(1,3)
(y) | mat.at<type>(2,0) mat.at<type>(2,1) mat.at<type>(2,2) mat.at<type>(2,3)
    ▼
```

Here:

* The **first index** is the row (vertical, y),
* The **second index** is the column (horizontal, x).

---



| Type            | Access Form                             | Equivalent   |
| --------------- | --------------------------------------- | ------------ |
| Matrix indexing | `img.at<uchar>(row, column)`            |              |
| Geometric point | `img.at<uchar>(cv::Point(column, row))` | ✅ same pixel |

So:

```cpp
std::cout << static_cast<unsigned>(img.at<uchar>(row, column)) << std::endl;
std::cout << static_cast<unsigned>(img.at<uchar>(cv::Point(column, row))) << std::endl;
```

will always print **the same value**.

---

####  Quick Practical Rule of Thumb

| API Type                               | Coordinate Order           | Example                              |
| -------------------------------------- | -------------------------- | ------------------------------------ |
| **Pixel access**                       | `(row, column)` = `(y, x)` | `img.at<Vec3b>(y, x)`                |
| **Geometry & Drawing**                 | `(x, y)`                   | `cv::circle(img, Point(x, y), ...)`  |
| **Transformation (warpAffine, remap)** | `(x=u, y=v)`               | consistent with geometric convention |

---


> In OpenCV:
> **Matrix indices are (row, column) = (y, x)**
> **Geometric functions use Point(x, y) = (column, row)**
> Both point to the same pixel, but the argument order flips.

---


### **Numerical Example**

####  **Vertical Flip**

**Source Image**

Let’s define a small image:

$$
I_s =
\begin{bmatrix}
10 & 20 & 30 \\
40 & 50 & 60 \\
70 & 80 & 90
\end{bmatrix}
$$

Each element is a pixel intensity.

The pixel coordinates are:

| $(row, col)$ | $(y, x)$    | $Value$ |
| ---------- | --------- | ----- |
| (0,0)      | (y=0,x=0) | 10    |
| (0,1)      | (0,1)     | 20    |
| (0,2)      | (0,2)     | 30    |
| (1,0)      | (1,0)     | 40    |
| (1,1)      | (1,1)     | 50    |
| (1,2)      | (1,2)     | 60    |
| (2,0)      | (2,0)     | 70    |
| (2,1)      | (2,1)     | 80    |
| (2,2)      | (2,2)     | 90    |



```cpp
              u (x = column)
    ------------------------------------------►
    | (0,0) (1,0) (2,0) (3,0)
    | (0,1) (1,1) (2,1) (3,1)
    | (0,2) (1,2) (2,2) (3,2)
  v | 
    |
    ▼
      (y = row)
```




Destination Image



$$
I_d =
\begin{bmatrix}
70 & 80 & 90 \\
40 & 50 & 60 \\
10 & 20 & 30
\end{bmatrix}
$$

---

#### Goal: Flip Image Vertically

We want the output image $ I_d $ to be **vertically flipped**, meaning:

$$
I_d(y, x) = I_s(2 - y, x)
$$

So row 0 in the destination comes from row 2 in source, etc.

---

#### Construct the Mapping Matrices

We must define two matrices, `map_x` and `map_y`, both of size $3 \times 3$:

* `map_x[y, x]` → x-coordinate in source
* `map_y[y, x]` → y-coordinate in source

#### For this transformation:

$$
map_x(y, x) = x
$$
$$
map_y(y, x) = 2 - y
$$

---

#### a) `map_x` (same as column indices)

$$
map_x =
\begin{bmatrix}
0 & 1 & 2 \\
0 & 1 & 2 \\
0 & 1 & 2
\end{bmatrix}
$$

#### b) `map_y` (flipped vertically)

$$
map_y =
\begin{bmatrix}
2 & 2 & 2 \\
1 & 1 & 1 \\
0 & 0 & 0
\end{bmatrix}
$$

---

#### Compute Output Pixel Values

For each pixel (y, x) in the **destination**, we sample:

$$
I_d(y, x) = I_s( map_y(y,x),\ map_x(y,x) )
$$

| $(y,x)$ | $(map_y, map_x)$ | Sampled Value | Explanation             |
| ----- | -------------- | ------------- | ----------------------- |
| (0,0) | (2,0)          | 70            | top row from bottom row |
| (0,1) | (2,1)          | 80            |                         |
| (0,2) | (2,2)          | 90            |                         |
| (1,0) | (1,0)          | 40            | middle stays            |
| (1,1) | (1,1)          | 50            |                         |
| (1,2) | (1,2)          | 60            |                         |
| (2,0) | (0,0)          | 10            | bottom row from top row |
| (2,1) | (0,1)          | 20            |                         |
| (2,2) | (0,2)          | 30            |                         |

---

#### 5. Output Image

Thus:

$$
I_d =
\begin{bmatrix}
70 & 80 & 90 \\
40 & 50 & 60 \\
10 & 20 & 30
\end{bmatrix}
$$

Exactly a **vertical flip** of the source image.

---

#### 6. OpenCV Equivalent Code


In [3]:

import cv2
import numpy as np

# Define source image
src = np.array([[10, 20, 30],
                [40, 50, 60],
                [70, 80, 90]], dtype=np.uint8)

h, w = src.shape

# Create mapping
map_x = np.zeros((h, w), np.float32)
map_y = np.zeros((h, w), np.float32)

for y in range(h):
    for x in range(w):
        map_x[y, x] = x
        map_y[y, x] = h - 1 - y   # flip vertically

# Apply remap
dst = cv2.remap(src, map_x, map_y, interpolation=cv2.INTER_NEAREST)

print("Source:\n", src)
print("map_x:\n", map_x)
print("map_y:\n", map_y)
print("Destination:\n", dst)


Source:
 [[10 20 30]
 [40 50 60]
 [70 80 90]]
map_x:
 [[0. 1. 2.]
 [0. 1. 2.]
 [0. 1. 2.]]
map_y:
 [[2. 2. 2.]
 [1. 1. 1.]
 [0. 0. 0.]]
Destination:
 [[70 80 90]
 [40 50 60]
 [10 20 30]]


---


| Matrix  | Meaning                                | Example (3×3 case)                              |
| ------- | -------------------------------------- | ----------------------------------------------- |
| `map_x` | X-coordinate in source                 | $\begin{bmatrix}0&1&2\\ 0&1&2\\ 0&1&2 \end{bmatrix}$ |
| `map_y` | Y-coordinate in source                 | $\begin{bmatrix}2&2&2\\ 1&1&1\\ 0&0&0 \end{bmatrix}$ |
| Output  | $I_d(y,x)=I_s(map_y(y,x), map_x(y,x))$ | Flipped vertically                              |

---


####  **Horizontal Flip**

**Horizontal Flip**

We want the output image to look like this:

```
| (2,0) | (1,0) | (0,0) |
| (2,1) | (1,1) | (0,1) |
| (2,2) | (1,2) | (0,2) |
```
In words:
the left–right order is reversed,
so pixel `(u,v)` in the destination should come from `(x_src, y_src)` = `(width−1−u, v)`.

---

#### Compute the Mapping

For image width = 3,
we have `width - 1 = 2`.

Then:

$$
map_x(v, u) = 2 - u \
map_y(v, u) = v
$$

---

#### Fill the Maps

**`map_x`**

| v\→u | 0 | 1 | 2 |
| ---- | - | - | - |
| 0    | 2 | 1 | 0 |
| 1    | 2 | 1 | 0 |
| 2    | 2 | 1 | 0 |

**`map_y`**

| v\→u | 0 | 1 | 2 |
| ---- | - | - | - |
| 0    | 0 | 0 | 0 |
| 1    | 1 | 1 | 1 |
| 2    | 2 | 2 | 2 |


---

#### Check Pixel by Pixel

Let’s verify:

| Output (u,v) | Source (map_x, map_y) | Source pixel |
| ------------ | --------------------- | ------------ |
| (0,0)        | (2,0)                 | (2,0)        |
| (1,0)        | (1,0)                 | (1,0)        |
| (2,0)        | (0,0)                 | (0,0)        |
| (0,1)        | (2,1)                 | (2,1)        |
| (1,1)        | (1,1)                 | (1,1)        |
| (2,1)        | (0,1)                 | (0,1)        |
| (0,2)        | (2,2)                 | (2,2)        |
| (1,2)        | (1,2)                 | (1,2)        |
| (2,2)        | (0,2)                 | (0,2)        |

every output pixel correctly samples the **mirrored** source.

---




Excellent — now let’s extend our previous **3×3 numerical example** to a **rotation case** to see how `cv::remap` works with **fractional coordinates** and **interpolation**.

---

## 1. Source Image

We’ll use the same 3×3 grayscale image:

$$
I_s =
\begin{bmatrix}
10 & 20 & 30 \
40 & 50 & 60 \
70 & 80 & 90
\end{bmatrix}
$$

---

## 2. Goal: Rotate Image 90° Counterclockwise About Its Center

We’ll compute `map_x` and `map_y` manually.

Let the rotation matrix be:

$$
R =
\begin{bmatrix}
\cos\theta & -\sin\theta \
\sin\theta &  \cos\theta
\end{bmatrix}
$$

For ( \theta = +90° ):

$$
R =
\begin{bmatrix}
0 & -1 \
1 &  0
\end{bmatrix}
$$

---

## 3. Compute the Transformation Center

Image width ( w = 3 ), height ( h = 3 )

Center coordinates:
$$
c_x = (w - 1)/2 = 1 \
c_y = (h - 1)/2 = 1
$$

---

## 4. Mapping Rule (Inverse Mapping)

Remember: **`remap` uses inverse mapping** — for each destination pixel ( (u,v) ), we find its source coordinates ( (x,y) ).

If the forward rotation is:

$$
\begin{bmatrix} u \ v \end{bmatrix}
===================================

R
\begin{bmatrix} x - c_x \ y - c_y \end{bmatrix}
+
\begin{bmatrix} c_x \ c_y \end{bmatrix}
$$

then the inverse is:

$$
\begin{bmatrix} x \ y \end{bmatrix}
===================================

R^{-1}
\begin{bmatrix} u - c_x \ v - c_y \end{bmatrix}
+
\begin{bmatrix} c_x \ c_y \end{bmatrix}
$$

Since (R^{-1} = R^T), and (R^T = \begin{bmatrix} 0 & 1 \ -1 & 0 \end{bmatrix}), we have:

$$
x = c_x + (v - c_y)
$$
$$
y = c_y - (u - c_x)
$$

---

## 5. Compute `map_x` and `map_y`

We’ll compute them for all pixel coordinates ( (u,v) ) in the destination.

| (v,u) | (x = 1 + (v-1)) | (y = 1 - (u-1)) | `map_x` | `map_y` |
| ----- | --------------- | --------------- | ------- | ------- |
| (0,0) | 0               | 2               | 0       | 2       |
| (0,1) | 0               | 1               | 1       | 2       |
| (0,2) | 0               | 0               | 2       | 2       |
| (1,0) | 1               | 2               | 0       | 1       |
| (1,1) | 1               | 1               | 1       | 1       |
| (1,2) | 1               | 0               | 2       | 1       |
| (2,0) | 2               | 2               | 0       | 0       |
| (2,1) | 2               | 1               | 1       | 0       |
| (2,2) | 2               | 0               | 2       | 0       |

---

### `map_x`

(remember: `map_x` corresponds to **x** in source)

$$
map_x =
\begin{bmatrix}
0 & 0 & 0 \
1 & 1 & 1 \
2 & 2 & 2
\end{bmatrix}
$$

### `map_y`

(remember: `map_y` corresponds to **y** in source)

$$
map_y =
\begin{bmatrix}
2 & 1 & 0 \
2 & 1 & 0 \
2 & 1 & 0
\end{bmatrix}
$$

---

## 6. Compute Output Image

For each pixel `(v,u)` in destination:

$$
I_d(v,u) = I_s(map_y(v,u),\ map_x(v,u))
$$

| (v,u) | (map_y,map_x) | Source Value |                        |
| ----- | ------------- | ------------ | ---------------------- |
| (0,0) | (2,0)         | 70           | top-left ← bottom-left |
| (0,1) | (1,0)         | 40           |                        |
| (0,2) | (0,0)         | 10           |                        |
| (1,0) | (2,1)         | 80           |                        |
| (1,1) | (1,1)         | 50           |                        |
| (1,2) | (0,1)         | 20           |                        |
| (2,0) | (2,2)         | 90           |                        |
| (2,1) | (1,2)         | 60           |                        |
| (2,2) | (0,2)         | 30           |                        |

---

## 7. Resulting Output Image

$$
I_d =
\begin{bmatrix}
70 & 40 & 10 \
80 & 50 & 20 \
90 & 60 & 30
\end{bmatrix}
$$

✅ This is indeed the source rotated **90° counterclockwise**.

---

## 8. Code Example

```python
import cv2
import numpy as np

src = np.array([[10, 20, 30],
                [40, 50, 60],
                [70, 80, 90]], dtype=np.uint8)

h, w = src.shape
cx, cy = (w - 1)/2, (h - 1)/2

map_x = np.zeros((h, w), np.float32)
map_y = np.zeros((h, w), np.float32)

for v in range(h):
    for u in range(w):
        map_x[v, u] = cx + (v - cy)    # x = cx + (v - cy)
        map_y[v, u] = cy - (u - cx)    # y = cy - (u - cx)

dst = cv2.remap(src, map_x, map_y, interpolation=cv2.INTER_NEAREST)

print("Source:\n", src)
print("map_x:\n", map_x)
print("map_y:\n", map_y)
print("Destination:\n", dst)
```

Output:

```
Source:
[[10 20 30]
 [40 50 60]
 [70 80 90]]

map_x:
[[0. 0. 0.]
 [1. 1. 1.]
 [2. 2. 2.]]

map_y:
[[2. 1. 0.]
 [2. 1. 0.]
 [2. 1. 0.]]

Destination:
[[70 40 10]
 [80 50 20]
 [90 60 30]]
```

---

## 9. Observations

1. When `map_x`/`map_y` are **integer coordinates**, interpolation has no effect (same as `INTER_NEAREST`).
2. For **fractional coordinates** (e.g., rotation by 45°), `cv::remap` performs **bilinear interpolation**, mixing neighboring pixel intensities.

---

Would you like me to extend this rotation example to **45° rotation** next — so you can see how fractional coordinates produce interpolation values?



## Signature and Input Parameters

From the tutorial, the C++ signature is:

```cpp
void remap(
    InputArray src,
    OutputArray dst,
    InputArray map_x,
    InputArray map_y,
    int interpolation,
    int borderMode = BORDER_CONSTANT,
    const Scalar &borderValue = Scalar()
);
```

Here are the parameters:

| Parameter       | Meaning                                                    | Requirements / Notes                                                                                            |
| --------------- | ---------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------- |
| `src`           | Input (source) image                                       | Must be a valid image (e.g. single or multi-channel)                                                            |
| `dst`           | Output (destination) image                                 | Same size (rows × cols) as mapping arrays; same type as `src`                                                   |
| `map_x`         | Horizontal mapping                                         | A floating-point array (same size as `dst`) such that `map_x(u, v)` is the x-coordinate in `src` to sample from |
| `map_y`         | Vertical mapping                                           | A floating-point array (same size as `dst`) such that `map_y(u, v)` is the y-coordinate in `src` to sample from |
| `interpolation` | Interpolation method                                       | E.g. `INTER_NEAREST`, `INTER_LINEAR`, `INTER_CUBIC`                                                             |
| `borderMode`    | How to treat pixels whose mapping falls outside the source | E.g. `BORDER_CONSTANT`, `BORDER_REPLICATE`, `BORDER_REFLECT`, etc.                                              |
| `borderValue`   | Value used when `borderMode = BORDER_CONSTANT`             | A constant color (per channel) for out-of-bounds areas                                                          |

In Python (cv2), it’s:

```python
dst = cv2.remap(src, map_x, map_y, interpolation, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
```

---



## **When and why you use `cv::remap`**
for *simple, global* transformations (rotation, scaling, translation, affine, perspective), functions like
`cv::getRotationMatrix2D`, `cv::warpAffine`, or `cv::warpPerspective` are indeed **simpler and faster** than manually using `cv::remap`.


---

### Difference in Purpose

| Function          | Type of transformation          | Parameters                          | Typical use                             |
| ----------------- | ------------------------------- | ----------------------------------- | --------------------------------------- |
| `warpAffine`      | 2×3 **linear affine**           | rotation, scale, translation, shear | Global linear transforms                |
| `warpPerspective` | 3×3 **homography**              | perspective mapping                 | Planar projection (e.g. rectification)  |
| `remap`           | **arbitrary per-pixel mapping** | two float maps (`map_x`, `map_y`)   | Nonlinear, spatially varying transforms |

 **Key idea:**
`warpAffine` and `warpPerspective` are *special cases* of `remap` where the mapping is globally linear.
`remap` gives you **full control** over each pixel — you can warp images in any nonlinear way.

---

###  When to Use `remap`



#### **(1) Lens Distortion Correction**

Cameras produce distorted images due to lens geometry.
The mapping from distorted → undistorted pixels is nonlinear, depending on radial/tangential distortion coefficients.

You can precompute the mapping using:

```cpp
cv::initUndistortRectifyMap(K, D, R, P, image_size, CV_32FC1, map_x, map_y);
cv::remap(src, dst, map_x, map_y, cv::INTER_LINEAR);
```

`remap` is perfect here because each pixel’s mapping is unique and nonlinear.

---

#### **(2) Image Rectification (Stereo Calibration)**

After calibrating stereo cameras, you “rectify” them so epipolar lines align horizontally.
This is also a nonlinear per-pixel transformation — `remap` is used to apply the rectification maps.

```cpp
cv::initUndistortRectifyMap(...);
cv::remap(left_img, rect_left, map1x, map1y, INTER_LINEAR);
cv::remap(right_img, rect_right, map2x, map2y, INTER_LINEAR);
```

---

#### **(3) Optical Flow Warping**

If you have optical flow fields `(u,v)` between two frames, you can use them to warp one frame to the other.

```cpp
map_x = np.arange(w)[None, :] + flow[..., 0]
map_y = np.arange(h)[:, None] + flow[..., 1]
dst = cv2.remap(src, map_x, map_y, cv2.INTER_LINEAR)
```

This creates smooth motion compensation or frame interpolation.

---

#### **(4) Custom Image Warps**

You might design a custom warp, e.g.:

* polar-to-cartesian coordinate mapping
* fish-eye panoramic unwrapping
* cylindrical / spherical projection
* bending / twisting / wave distortions

All of these require per-pixel control → only possible via `remap`.

Example:

```cpp
for (int y = 0; y < rows; y++)
  for (int x = 0; x < cols; x++) {
      float r = hypot(x - cx, y - cy);
      float theta = atan2(y - cy, x - cx);
      map_x.at<float>(y, x) = theta * scale_theta + offset_x;
      map_y.at<float>(y, x) = r * scale_r + offset_y;
  }
```

---

#### **(5) Lookup-Based or Precomputed Mapping**

If you need to apply the **same spatial transform many times**,
you can precompute `map_x` and `map_y` once, then reuse them.

Example:

```cpp
cv::remap(frame, corrected, map_x, map_y, cv::INTER_LINEAR);
```

Fast and efficient in real-time camera pipelines.

---

## When *Not* to Use `remap`

Use `warpAffine` or `warpPerspective` when:

* You’re only rotating, scaling, or translating the whole image (single matrix transform).
* You can express your mapping with a **global affine/homography**.
* You want a simple transformation that can be expressed as `dst = A * src`.

These functions are simpler and a bit faster because OpenCV internally optimizes them for matrix-based warps.

---

## 4. Summary Table

| Task                                       | Transformation Type   | Recommended Function                |
| ------------------------------------------ | --------------------- | ----------------------------------- |
| Rotate / scale / translate                 | Linear                | `warpAffine`, `getRotationMatrix2D` |
| Perspective transform                      | Projective            | `warpPerspective`                   |
| Lens undistortion                          | Nonlinear             | `remap`                             |
| Stereo rectification                       | Nonlinear             | `remap`                             |
| Optical flow warping                       | Nonlinear             | `remap`                             |
| Custom polar / fisheye / cylindrical warps | Nonlinear             | `remap`                             |
| Real-time correction (repeated transform)  | Precomputed nonlinear | `remap`                             |

---

